In [413]:
import pandas as pd
import numpy as np

In [414]:
df = pd.read_csv('theta_datalog.csv')
df.head()

,JOB_NAME,COBALT_JOBID,MACHINE_NAME,QUEUED_TIMESTAMP,QUEUED_DATE_ID,START_TIMESTAMP,START_DATE_ID,END_TIMESTAMP,END_DATE_ID,USERNAME_GENID,...,IS_SUBBLOCK,IS_SUBBLOCK_ONLY,IS_MULTILOCATION_ONLY,IS_MULTILOCATION_SUBBLOCK,IS_CONSECUTIVE_ONLY,IS_SINGLE_ONLY,IS_NO_TASKS,IS_OTHER,OVERBURN_CORE_HOURS,IS_OVERBURN
0,639487.theta,639487,theta,2022-12-31 23:28:21,20221231,2022-12-31 23:29:32,20221231,2023-01-01 00:30:05,20230101,6.387939e+13,...,0,0,0,0,0,0,0,0,0.0000,0
1,639448.theta,639448,theta,2022-12-31 17:30:46,20221231,2022-12-31 19:40:18,20221231,2023-01-01 01:41:13,20230101,NaN,...,0,0,0,0,0,0,0,0,0.0000,0
2,639135.theta,639135,theta,2022-12-27 21:00:46,20221227,2022-12-31 19:43:38,20221231,2023-01-01 01:44:34,20230101,2.785630e+13,...,0,0,0,0,0,0,0,0,0.0000,0
3,638720.theta,638720,theta,2022-12-24 18:04:32,20221224,2023-01-01 01:46:34,20230101,2023-01-01 02:47:23,20230101,7.200789e+13,...,0,0,0,0,0,0,0,0,16489.0468,1
4,639463.theta,639463,theta,2022-12-31 19:12:30,20221231,2023-01-01 01:42:54,20230101,2023-01-01 03:22:49,20230101,NaN,...,0,0,0,0,0,0,0,0,0.0000,0


In [415]:
columns_names = df.columns
print(columns_names)
(df['CORES_USED'] == df['CORES_REQUESTED']).sum()
df['state'] = 0
df['gpu_num'] = 0
print(df['ELIGIBLE_WAIT_SECONDS'])
print(df['EXIT_STATUS'])

Index(['JOB_NAME', 'COBALT_JOBID', 'MACHINE_NAME', 'QUEUED_TIMESTAMP',
       'QUEUED_DATE_ID', 'START_TIMESTAMP', 'START_DATE_ID', 'END_TIMESTAMP',
       'END_DATE_ID', 'USERNAME_GENID', 'PROJECT_NAME_GENID', 'QUEUE_NAME',
       'WALLTIME_SECONDS', 'RUNTIME_SECONDS', 'NODES_USED', 'NODES_REQUESTED',
       'CORES_USED', 'CORES_REQUESTED', 'LOCATION', 'EXIT_STATUS',
       'ELIGIBLE_WAIT_SECONDS', 'ELIGIBLE_WAIT_FACTOR', 'QUEUED_WAIT_SECONDS',
       'QUEUED_WAIT_FACTOR', 'REQUESTED_CORE_HOURS', 'USED_CORE_HOURS',
       'CAPABILITY_USAGE_CORE_HOURS', 'NONCAPABILITY_USAGE_CORE_HOURS',
       'BUCKETS3_A_USAGE_CORE_HOURS', 'BUCKETS3_B_USAGE_CORE_HOURS',
       'BUCKETS3_C_USAGE_CORE_HOURS', 'MACHINE_PARTITION', 'EXIT_CODE', 'MODE',
       'RESID', 'DATA_LOAD_STATUS', 'CAPABILITY', 'SIZE_BUCKETS3',
       'PERCENTILE', 'NUM_TASKS_SUBBLOCK', 'NUM_TASKS_CONSECUTIVE',
       'NUM_TASKS_MULTILOCATION', 'NUM_TASKS_SINGLE', 'COBALT_NUM_TASKS',
       'IS_SINGLE', 'IS_CONSECUTIVE', 'IS_MULTIL

0        143
1          0
2        143
3        143
4          0
        ... 
23382      0
23383    143
23384      0
23385    143
23386    143
Name: EXIT_STATUS, Length: 23387, dtype: int64

In [416]:
df = df[['JOB_NAME', 'USERNAME_GENID', 'PROJECT_NAME_GENID', 'state', 'gpu_num', 'CORES_USED', 'NODES_USED', 'QUEUED_TIMESTAMP', 'START_TIMESTAMP', 'RUNTIME_SECONDS', 'WALLTIME_SECONDS', 'EXIT_STATUS']]
df.columns

Index(['JOB_NAME', 'USERNAME_GENID', 'PROJECT_NAME_GENID', 'state', 'gpu_num',
       'CORES_USED', 'NODES_USED', 'QUEUED_TIMESTAMP', 'START_TIMESTAMP',
       'RUNTIME_SECONDS', 'WALLTIME_SECONDS', 'EXIT_STATUS'],
      dtype='object')

In [417]:
df.head()

,JOB_NAME,USERNAME_GENID,PROJECT_NAME_GENID,state,gpu_num,CORES_USED,NODES_USED,QUEUED_TIMESTAMP,START_TIMESTAMP,RUNTIME_SECONDS,WALLTIME_SECONDS,EXIT_STATUS
0,639487.theta,6.387939e+13,NaN,0,0,512.0,8.0,2022-12-31 23:28:21,2022-12-31 23:29:32,3632.92,3600.0,143
1,639448.theta,NaN,NaN,0,0,16384.0,256.0,2022-12-31 17:30:46,2022-12-31 19:40:18,21654.66,21600.0,0
2,639135.theta,2.785630e+13,NaN,0,0,262144.0,4096.0,2022-12-27 21:00:46,2022-12-31 19:43:38,21655.97,21600.0,143
3,638720.theta,7.200789e+13,7.029633e+13,0,0,262144.0,4096.0,2022-12-24 18:04:32,2023-01-01 01:46:34,3648.70,3600.0,143
4,639463.theta,NaN,NaN,0,0,8192.0,128.0,2022-12-31 19:12:30,2023-01-01 01:42:54,5995.18,10800.0,0


In [418]:
import datetime

def string_to_timestamp(time_string):
    dt_object = datetime.datetime.strptime(time_string, '%Y-%m-%d %H:%M:%S')
    return int(dt_object.timestamp())

df['QUEUED_TIMESTAMP'] = df['QUEUED_TIMESTAMP'].apply(string_to_timestamp)
df['START_TIMESTAMP'] = df['START_TIMESTAMP'].apply(string_to_timestamp)

In [419]:
df.head()

,JOB_NAME,USERNAME_GENID,PROJECT_NAME_GENID,state,gpu_num,CORES_USED,NODES_USED,QUEUED_TIMESTAMP,START_TIMESTAMP,RUNTIME_SECONDS,WALLTIME_SECONDS,EXIT_STATUS
0,639487.theta,6.387939e+13,NaN,0,0,512.0,8.0,1672529301,1672529372,3632.92,3600.0,143
1,639448.theta,NaN,NaN,0,0,16384.0,256.0,1672507846,1672515618,21654.66,21600.0,0
2,639135.theta,2.785630e+13,NaN,0,0,262144.0,4096.0,1672174846,1672515818,21655.97,21600.0,143
3,638720.theta,7.200789e+13,7.029633e+13,0,0,262144.0,4096.0,1671905072,1672537594,3648.70,3600.0,143
4,639463.theta,NaN,NaN,0,0,8192.0,128.0,1672513950,1672537374,5995.18,10800.0,0


In [420]:
df['PROJECT_NAME_GENID'] = df['PROJECT_NAME_GENID'].fillna(0)
df['USERNAME_GENID'] = df['PROJECT_NAME_GENID'].fillna(0)
df['state'] = df['state'].fillna(0)
df['gpu_num'] = df['gpu_num'].fillna(0)


In [421]:
df.head()

,JOB_NAME,USERNAME_GENID,PROJECT_NAME_GENID,state,gpu_num,CORES_USED,NODES_USED,QUEUED_TIMESTAMP,START_TIMESTAMP,RUNTIME_SECONDS,WALLTIME_SECONDS,EXIT_STATUS
0,639487.theta,0.000000e+00,0.000000e+00,0,0,512.0,8.0,1672529301,1672529372,3632.92,3600.0,143
1,639448.theta,0.000000e+00,0.000000e+00,0,0,16384.0,256.0,1672507846,1672515618,21654.66,21600.0,0
2,639135.theta,0.000000e+00,0.000000e+00,0,0,262144.0,4096.0,1672174846,1672515818,21655.97,21600.0,143
3,638720.theta,7.029633e+13,7.029633e+13,0,0,262144.0,4096.0,1671905072,1672537594,3648.70,3600.0,143
4,639463.theta,0.000000e+00,0.000000e+00,0,0,8192.0,128.0,1672513950,1672537374,5995.18,10800.0,0


In [422]:
df2 = pd.read_csv('data_supercloud.csv')
df2.head()

,job,user,project,state,gpu_num,cpu_num,node_num,submit_time,wait_time,run_time,wall_time,node_hour,new_status
0,8.270000e+13,4.280000e+13,0,3,0,20,1,1609535703,270594,308,0,0.085556,pass
1,3.430000e+12,4.280000e+13,0,3,0,20,1,1609535703,270904,397,0,0.110278,pass
2,8.370000e+12,4.280000e+13,0,3,0,20,1,1609535703,271301,327,0,0.090833,pass
3,5.060000e+13,2.810000e+13,0,5,0,1,1,1609807695,0,3851,0,1.069722,failed
4,1.610000e+13,4.280000e+13,0,3,0,20,1,1609535912,272558,307,0,0.085278,pass


In [423]:
df['JOB_NAME'] = df['JOB_NAME'].str.split('.').str[0]
df.head()

,JOB_NAME,USERNAME_GENID,PROJECT_NAME_GENID,state,gpu_num,CORES_USED,NODES_USED,QUEUED_TIMESTAMP,START_TIMESTAMP,RUNTIME_SECONDS,WALLTIME_SECONDS,EXIT_STATUS
0,639487,0.000000e+00,0.000000e+00,0,0,512.0,8.0,1672529301,1672529372,3632.92,3600.0,143
1,639448,0.000000e+00,0.000000e+00,0,0,16384.0,256.0,1672507846,1672515618,21654.66,21600.0,0
2,639135,0.000000e+00,0.000000e+00,0,0,262144.0,4096.0,1672174846,1672515818,21655.97,21600.0,143
3,638720,7.029633e+13,7.029633e+13,0,0,262144.0,4096.0,1671905072,1672537594,3648.70,3600.0,143
4,639463,0.000000e+00,0.000000e+00,0,0,8192.0,128.0,1672513950,1672537374,5995.18,10800.0,0


In [424]:
# Can be used for wait time
df['wait_time'] = df['START_TIMESTAMP']-df['QUEUED_TIMESTAMP']
df.head()

,JOB_NAME,USERNAME_GENID,PROJECT_NAME_GENID,state,gpu_num,CORES_USED,NODES_USED,QUEUED_TIMESTAMP,START_TIMESTAMP,RUNTIME_SECONDS,WALLTIME_SECONDS,EXIT_STATUS,wait_time
0,639487,0.000000e+00,0.000000e+00,0,0,512.0,8.0,1672529301,1672529372,3632.92,3600.0,143,71
1,639448,0.000000e+00,0.000000e+00,0,0,16384.0,256.0,1672507846,1672515618,21654.66,21600.0,0,7772
2,639135,0.000000e+00,0.000000e+00,0,0,262144.0,4096.0,1672174846,1672515818,21655.97,21600.0,143,340972
3,638720,7.029633e+13,7.029633e+13,0,0,262144.0,4096.0,1671905072,1672537594,3648.70,3600.0,143,632522
4,639463,0.000000e+00,0.000000e+00,0,0,8192.0,128.0,1672513950,1672537374,5995.18,10800.0,0,23424


In [425]:
len(df)

23387

In [426]:
print(df['NODES_USED'].dtypes)
print(df['RUNTIME_SECONDS'].dtypes)

df['node_hour'] = df['NODES_USED'] * df['RUNTIME_SECONDS']

float64
float64


In [427]:
df.head()

,JOB_NAME,USERNAME_GENID,PROJECT_NAME_GENID,state,gpu_num,CORES_USED,NODES_USED,QUEUED_TIMESTAMP,START_TIMESTAMP,RUNTIME_SECONDS,WALLTIME_SECONDS,EXIT_STATUS,wait_time,node_hour
0,639487,0.000000e+00,0.000000e+00,0,0,512.0,8.0,1672529301,1672529372,3632.92,3600.0,143,71,29063.36
1,639448,0.000000e+00,0.000000e+00,0,0,16384.0,256.0,1672507846,1672515618,21654.66,21600.0,0,7772,5543592.96
2,639135,0.000000e+00,0.000000e+00,0,0,262144.0,4096.0,1672174846,1672515818,21655.97,21600.0,143,340972,88702853.12
3,638720,7.029633e+13,7.029633e+13,0,0,262144.0,4096.0,1671905072,1672537594,3648.70,3600.0,143,632522,14945075.20
4,639463,0.000000e+00,0.000000e+00,0,0,8192.0,128.0,1672513950,1672537374,5995.18,10800.0,0,23424,767383.04


In [428]:
df['node_hour'] = df['node_hour']/3600

In [429]:
df.head()

,JOB_NAME,USERNAME_GENID,PROJECT_NAME_GENID,state,gpu_num,CORES_USED,NODES_USED,QUEUED_TIMESTAMP,START_TIMESTAMP,RUNTIME_SECONDS,WALLTIME_SECONDS,EXIT_STATUS,wait_time,node_hour
0,639487,0.000000e+00,0.000000e+00,0,0,512.0,8.0,1672529301,1672529372,3632.92,3600.0,143,71,8.073156
1,639448,0.000000e+00,0.000000e+00,0,0,16384.0,256.0,1672507846,1672515618,21654.66,21600.0,0,7772,1539.886933
2,639135,0.000000e+00,0.000000e+00,0,0,262144.0,4096.0,1672174846,1672515818,21655.97,21600.0,143,340972,24639.681422
3,638720,7.029633e+13,7.029633e+13,0,0,262144.0,4096.0,1671905072,1672537594,3648.70,3600.0,143,632522,4151.409778
4,639463,0.000000e+00,0.000000e+00,0,0,8192.0,128.0,1672513950,1672537374,5995.18,10800.0,0,23424,213.161956


In [430]:
df['EXIT_STATUS'].unique()

array([143,   0, 137,   1, 127, 255,   8,   3, 134,  19,  24,   2, 130,
       135, 136, 139, 174, 126, 168,  53,  74,  10, 256, 254,  59, 120])

In [431]:
def code_to_status(code):
  if code == 0:
    return 'Pass'
  elif code == 137:
    return 'Killed'
  elif code == 143:
    return 'Killed'
  else:
    return 'Failed'

df['new_status'] = df['EXIT_STATUS'].apply(code_to_status)


In [432]:
df['new_status'].unique()

array(['Killed', 'Pass', 'Failed'], dtype=object)

In [433]:
df.head()

,JOB_NAME,USERNAME_GENID,PROJECT_NAME_GENID,state,gpu_num,CORES_USED,NODES_USED,QUEUED_TIMESTAMP,START_TIMESTAMP,RUNTIME_SECONDS,WALLTIME_SECONDS,EXIT_STATUS,wait_time,node_hour,new_status
0,639487,0.000000e+00,0.000000e+00,0,0,512.0,8.0,1672529301,1672529372,3632.92,3600.0,143,71,8.073156,Killed
1,639448,0.000000e+00,0.000000e+00,0,0,16384.0,256.0,1672507846,1672515618,21654.66,21600.0,0,7772,1539.886933,Pass
2,639135,0.000000e+00,0.000000e+00,0,0,262144.0,4096.0,1672174846,1672515818,21655.97,21600.0,143,340972,24639.681422,Killed
3,638720,7.029633e+13,7.029633e+13,0,0,262144.0,4096.0,1671905072,1672537594,3648.70,3600.0,143,632522,4151.409778,Killed
4,639463,0.000000e+00,0.000000e+00,0,0,8192.0,128.0,1672513950,1672537374,5995.18,10800.0,0,23424,213.161956,Pass


In [434]:
df = df[['JOB_NAME', 'USERNAME_GENID', 'PROJECT_NAME_GENID', 'state', 'gpu_num', 'CORES_USED', 'NODES_USED', 'QUEUED_TIMESTAMP', 'wait_time', 'RUNTIME_SECONDS', 'WALLTIME_SECONDS', 'node_hour', 'new_status']]
df.head()

,JOB_NAME,USERNAME_GENID,PROJECT_NAME_GENID,state,gpu_num,CORES_USED,NODES_USED,QUEUED_TIMESTAMP,wait_time,RUNTIME_SECONDS,WALLTIME_SECONDS,node_hour,new_status
0,639487,0.000000e+00,0.000000e+00,0,0,512.0,8.0,1672529301,71,3632.92,3600.0,8.073156,Killed
1,639448,0.000000e+00,0.000000e+00,0,0,16384.0,256.0,1672507846,7772,21654.66,21600.0,1539.886933,Pass
2,639135,0.000000e+00,0.000000e+00,0,0,262144.0,4096.0,1672174846,340972,21655.97,21600.0,24639.681422,Killed
3,638720,7.029633e+13,7.029633e+13,0,0,262144.0,4096.0,1671905072,632522,3648.70,3600.0,4151.409778,Killed
4,639463,0.000000e+00,0.000000e+00,0,0,8192.0,128.0,1672513950,23424,5995.18,10800.0,213.161956,Pass


In [435]:
df.columns = ['job', 'user', 'project', 'state', 'gpu_num', 'cpu_num', 'node_num', 'submit_time', 'wait_time', 'run_time', 'wall_time', 'node_hour', 'new_status']

In [436]:
df.head()

,job,user,project,state,gpu_num,cpu_num,node_num,submit_time,wait_time,run_time,wall_time,node_hour,new_status
0,639487,0.000000e+00,0.000000e+00,0,0,512.0,8.0,1672529301,71,3632.92,3600.0,8.073156,Killed
1,639448,0.000000e+00,0.000000e+00,0,0,16384.0,256.0,1672507846,7772,21654.66,21600.0,1539.886933,Pass
2,639135,0.000000e+00,0.000000e+00,0,0,262144.0,4096.0,1672174846,340972,21655.97,21600.0,24639.681422,Killed
3,638720,7.029633e+13,7.029633e+13,0,0,262144.0,4096.0,1671905072,632522,3648.70,3600.0,4151.409778,Killed
4,639463,0.000000e+00,0.000000e+00,0,0,8192.0,128.0,1672513950,23424,5995.18,10800.0,213.161956,Pass


In [437]:
df.to_csv('data_theta.csv')

In [438]:
df3 = pd.read_csv('data_theta.csv')

In [439]:
df3.head()

,Unnamed: 0,job,user,project,state,gpu_num,cpu_num,node_num,submit_time,wait_time,run_time,wall_time,node_hour,new_status
0,0,639487,0.000000e+00,0.000000e+00,0,0,512.0,8.0,1672529301,71,3632.92,3600.0,8.073156,Killed
1,1,639448,0.000000e+00,0.000000e+00,0,0,16384.0,256.0,1672507846,7772,21654.66,21600.0,1539.886933,Pass
2,2,639135,0.000000e+00,0.000000e+00,0,0,262144.0,4096.0,1672174846,340972,21655.97,21600.0,24639.681422,Killed
3,3,638720,7.029633e+13,7.029633e+13,0,0,262144.0,4096.0,1671905072,632522,3648.70,3600.0,4151.409778,Killed
4,4,639463,0.000000e+00,0.000000e+00,0,0,8192.0,128.0,1672513950,23424,5995.18,10800.0,213.161956,Pass


In [441]:
from IPython.display import FileLink
FileLink('data_theta.csv')

/content/data_theta.csv